# Patching with a view

We have sometimes the problem, that we get an OOM when we patch our arrays. So we can use 'view_as_windows' from skimage.util.shape to create an patched array that only use views from the original array.

In [1]:
import numpy as np
import os
import psutil

from skimage.util.shape import view_as_windows
from sklearn.feature_extraction import image
from datetime import datetime

In [2]:
# this function is to print the used memory
def get_used_memory(process_id: int, unit: str = "GB") -> str:
    unit = unit.upper()
    exponent = 0

    if unit == "KB":
        exponent = 1
    elif unit == "MB":
        exponent = 2
    elif unit == "GB":
        exponent = 3
    else:
        unit = "BYTES"

    process = psutil.Process(process_id)
    memory = process.memory_info().rss
    memory /= 1024 ** exponent
    return f"{round(memory, 3)} {unit}"

In [3]:
def get_pad_width(patch_size):
    pad = [int((s - 1) / 2) for s in patch_size]
    pad_width = [[pad[idx], pad[idx]] if s % 2 == 1 else [pad[idx], pad[idx] + 1] for idx, s in enumerate(patch_size)]
    pad_width.append([0, 0])

    return pad_width

In [4]:
def old_patch_function(spectrum: np.ndarray, patch_size):
    # Better not to use non even sizes
    pad_width = get_pad_width(patch_size=patch_size)
    spectrum_ = np.pad(array=spectrum, pad_width=np.array(pad_width))

    patches = image.extract_patches_2d(spectrum_, tuple(patch_size))
    patches = np.reshape(patches, (spectrum.shape[0], spectrum.shape[1], patch_size[0], patch_size[1], patches.shape[-1]))

    return patches

In [5]:
def new_patch_function(arr, patch_size, max_patches=None):
    """Extract patches from the input array as a view."""
    window_shape = (patch_size[0], patch_size[1], arr.shape[2] if len(arr.shape) > 2 else 1)
    patches = view_as_windows(arr, window_shape, step=1).squeeze()
    if max_patches is not None:
        patches = patches[:max_patches]
    return patches

In [6]:
def compare_functions(patch_size):
    process_id = os.getpid()

    size_ = [480, 640, 100]
    arr_ = np.arange(np.prod(size_)).reshape(size_)

    print(f"Memory only with original arrays: {get_used_memory(process_id)}")
    patch_shape_ = (5, 5)
    start_time_new = datetime.now()
    arr_pad = np.pad(arr_, get_pad_width(patch_size))
    print(f"Memory with original and pad arrays: {get_used_memory(process_id)}")
    patches_1 = new_patch_function(arr_pad, patch_size)
    print(f"Memory after new patch function: {get_used_memory(process_id)}")
    print(f"Execution time is: {datetime.now() - start_time_new}")
    print("Pad array an array with new pad function share memory:", np.shares_memory(arr_pad, patches_1))

    start_time_old = datetime.now()
    patches_2 = old_patch_function(arr_, patch_size)
    print(f"Memory after old patch function: {get_used_memory(process_id)}")
    print(f"Execution time is: {datetime.now() - start_time_old}")
    print("Array from new and old functon are the same:", np.all(patches_1 == patches_2))

In [7]:
test_sizes = [(3, 3), (5, 5), (7, 7), (9, 9)]

for size in test_sizes:
    print(f"Patch size: {size}")
    compare_functions(patch_size=size)
    print("")

Patch size: (3, 3)
Memory only with original arrays: 0.235 GB
Memory with original and pad arrays: 0.35 GB
Memory after new patch function: 0.35 GB
Execution time is: 0:00:00.117473
Pad array an array with new pad function share memory: True
Memory after old patch function: 1.38 GB
Execution time is: 0:00:01.056018
Array from new and old functon are the same: True

Patch size: (5, 5)
Memory only with original arrays: 0.235 GB
Memory with original and pad arrays: 0.351 GB
Memory after new patch function: 0.351 GB
Execution time is: 0:00:00.091104
Pad array an array with new pad function share memory: True
Memory after old patch function: 3.212 GB
Execution time is: 0:00:02.309445
Array from new and old functon are the same: True

Patch size: (7, 7)
Memory only with original arrays: 0.235 GB
Memory with original and pad arrays: 0.352 GB
Memory after new patch function: 0.352 GB
Execution time is: 0:00:00.088413
Pad array an array with new pad function share memory: True
Memory after old 